In [35]:
# determing the packet length for each rank
import math
import os
import numpy as np
# file_name = 'GCN_matrices/Cora_A.txt'
num_rank = 96
def get_packetLength_per_rank(num_rank): # This is the number of elements
    directory = 'GCN_matrices'
    dict_packet_length = {}
    dict_ratio = {}
    for filenm in os.listdir(directory): # iterate over all files
        file_name = os.path.join(directory, filenm)
        if os.path.isfile(file_name): # if it is a file
            splitted_file_name = file_name.split("/")[1].split("_")
            dataset = splitted_file_name[0]
#             print(file_name)
            bucket_list = [0 for g in range(num_rank)] # total: both diagonal and off-diagonal
            bucket_list_diagonal = [0 for g in range(num_rank)] # only diagonal
            with open(file_name) as f:
                line1 = f.readline() 
                line2 = int(f.readline())
                lines = f.readlines()
                num_row = int(line1.split()[0])
                num_col = int(line1.split()[1])
                bucket_length = int(num_row/num_rank)
                for i in range(line2):
#                     print(lines[i])
#                     print(i)
#                     print(row)
                    row = int(lines[i].split()[0])
                    col = int(lines[i].split()[1])
                    bucket = int(row/bucket_length)
                    if bucket > num_rank - 1: # if we are out of range as a result of rounding bucket_length \\
                        # just add these outliers to the last rank
                        bucket = num_rank - 1
                    bucket_list[bucket] += 1
                    if col >= bucket*bucket_length and col < (bucket +1) * bucket_length:
                        bucket_list_diagonal[bucket] += 1
            ratio_diag = np.array(bucket_list_diagonal)/np.array(bucket_list)
            # ratio of diag over total (both diagonal and off-diagonal)
            if splitted_file_name[1] == 'A.txt': # A matrix is being used for both layers
                key1 = dataset + '_L1_AXW'
                key2 = dataset + '_L2_AXW'
                dict_packet_length[key1] = bucket_list
                dict_packet_length[key2] = bucket_list
                dict_ratio[key1] = ratio_diag
                dict_ratio[key2] = ratio_diag
            elif splitted_file_name[1] == 'feat': 
                if splitted_file_name[2] == 'L1.txt':
                    key = dataset + '_L1_XW'
                    dict_packet_length[key] = bucket_list
                    dict_ratio[key] = ratio_diag
                elif splitted_file_name[2] == 'L2.txt':
                    key = dataset + '_L2_XW'
                    dict_packet_length[key] = bucket_list
                    dict_ratio[key] = ratio_diag
                else:
                    print('unsupported file name')
            else:
                print('unsupported file name')
    return dict_packet_length, dict_ratio
#     return dict_packet_length
# print(get_packetLength_per_rank(file_name, num_rank))

In [14]:
# dict_packet_length, ratio_diag = get_packetLength_per_rank(num_rank)
# print(dict_packet_length)
# print('-------------------------------------------')
# print(ratio_diag)

GCN_matrices/Cora_feat_L2.txt
GCN_matrices/Cora_A.txt
GCN_matrices/Cora_feat_L1.txt
GCN_matrices/Pubmed_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
{'Cora_L2_XW': [379, 366, 339, 392, 357, 366, 370, 390, 379, 385, 379, 369, 362, 382, 355, 381, 386, 370, 387, 368, 368, 354, 374, 385, 389, 385, 381, 367, 384, 378, 379, 388, 386, 377, 387, 375, 391, 381, 384, 381, 390, 381, 372, 352, 350, 379, 385, 373, 380, 380, 363, 369, 371, 380, 381, 390, 384, 384, 388, 374, 373, 392, 376, 363, 387, 385, 387, 382, 393, 382, 361, 350, 378, 389, 383, 389, 385, 387, 387, 386, 384, 389, 388, 395, 396, 365, 351, 387, 392, 371, 374, 387, 386, 388, 385, 643], 'Cora_L1_AXW': [94, 61, 50, 28, 80, 45, 32, 29, 62, 56, 71, 90, 74, 71, 87, 50, 62, 57, 65, 83, 77, 73, 67, 71, 67, 106, 88, 109, 111, 86, 85, 

In [9]:
from constant import *
import math
import random as ra


def sweep(sweep_threshold, sweep_step, payload_length, num_port):
    # payload_length is a list and is in Bytes
    ra.seed(1)
    results = []
    time_stamp = [0 for j in range(num_port)]
    sweep_num = int(sweep_threshold/sweep_step)
    num_expr = sweep_num ** num_port
    verbose_freq = 100 # showing the progress for each segment
    verbose_segment = int(num_expr/verbose_freq)
    verbose_num = 0
    for i in range(num_expr):
        # print(i)
        if i%verbose_segment == 0:
            print(verbose_num, " percent")
            verbose_num += 1
        # print(time_stamp)
        delay = comp_delay(time_stamp, payload_length, num_port, 'default', 0)
        results.append(delay)
        # print(delay)
        time_stamp[0] += sweep_step
        for k in range(num_port):
            if time_stamp[k] == sweep_threshold:
                time_stamp[k] = 0
                if k != num_port - 1:
                    time_stamp[k+1] += sweep_step
    return results

def sweepRand(num_expr, sweep_threshold, sweep_step, payload_length, num_port):
    # payload_length is a list and is in Bytes
    ra.seed(1)
    results = []
    sweep_num = int(sweep_threshold/sweep_step)
    verbose_freq = 100 # showing the progress for each segment
    verbose_segment = int(num_expr/verbose_freq)
    verbose_num = 0
    for i in range(num_expr):
        # print(i)
        if i%verbose_segment == 0:
            print(verbose_num, " percent")
            verbose_num += 1
        # print(time_stamp)
        # time_stamp[0] += sweep_step
        time_stamp = [ra.randrange(0, sweep_threshold, sweep_step) for j in range(num_port)]
        delay = comp_delay(time_stamp, payload_length, num_port, 'default', [0 for j in range(num_port)])
        results.append(delay)
        # print(delay)
    print('finished')
    return results

def comp_delay(time_stamp, payload_length, num_port, op_type, inp_packet_delay):
    # time is considered as ns
    # time_stamp is a list
    # payload_length is a list and is in Bytes
    # inp_packet_delay is a list
    mytime = 0
    end_time_port = [0 for i in range(num_port)]
    curr_queue = 0 # pointer to which queue
    num_beats = [math.ceil((payload_length[k] + HEADER_LENGTH)/PHIT_SIZE) for k in range(num_port)]
    if op_type == 'default':
        packet_delay = [num_beats[k] * PERIOD_CLK for k in range(num_port)]
    else:
        packet_delay = inp_packet_delay
    queuefinished = [False for i in range(num_port)]
    # num_finished = 0
    
    while all(queuefinished) == False:
        if mytime >= time_stamp[curr_queue] and queuefinished[curr_queue] == False: #schedule an consume the packet
            queuefinished[curr_queue] = True
            mytime += packet_delay[curr_queue]
            end_time_port[curr_queue] = mytime + SWITCH_LATENCY
            # print("queue id ", curr_queue, " is finished", "num_queue finished is: ", num_finished, "out of: ", num_port)
            # num_finished += 1
        
        # round-robin
        if (curr_queue == num_port - 1):
            curr_queue = 0
        else:
            curr_queue += 1
        
        # one clock delay for checking the next queue
        mytime += PERIOD_CLK 
    
    end_time_global = mytime + SWITCH_LATENCY
    return end_time_global, end_time_port
               
def analysis(results):
    # print(results)
    print('max: ', max(results))
    print('min: ', min(results))


In [ ]:
# num_port = 16
# results = sweepRand(num_expr = 1000000, sweep_threshold = 100, sweep_step = 10,
#                     payload_length = [1000 for k in range(num_port)], num_port = num_port)
# analysis(results)

In [33]:
from constant import *
def instruction_count(instr, vlen):
    if instr == 'vse32_v':
        return vlen + 6
    elif instr == 'vle32_v':
        return vlen + 6
    elif instr == 'vmacc_xv': # dense
        return vlen + PE_latency
    elif instr == 'vmacc_vx': # dense
        return vlen + PE_latency
    elif instr == 'spvmacc_xv': # sparse
        #     return vlen + PE_latency # lets factor out vlen and lump it with all off-diag nnz and instead only measure the overhead
        return 1 + PE_LATENCY # 1 is b/c of packet_length at the begining of packet
    elif instr == 'spvmacc_vx': # spearse
#         return vlen + PE_LATENCY # lets factor out vlen and lump it with all off-diag nnz and instead only measure the overhead
        return 1 + PE_LATENCY # 1 is b/c of packet_length at the begining of packet   
    elif instr == 'vstreamout_v':
        return vlen * NUM_COL
    elif instr == 'vsetivli':
        return 1
    elif instr == 'wfi':
        return 1
    elif (instr == 'addi' or instr == 'lui' or instr == 'add' or instr == 'bne'): # scalar
        return 1
    else:
        print('unsupported instruction')

def gcn_FPGA_time_cgra(instruction, packet_length, SIMD_under_utilization):
    num_rank = len(packet_length)
    cgra_time = [0 for i in range(num_rank)]
    for i in range(num_rank):
        if packet_length[i] == 0:
            cgra_time[i] = 0 # we should bypass cgra
        else:
            time_instr = 0 # init to zero
            for key in instruction:
                instr_vlen = instruction[key][0]
                instr_cnt = instruction[key][1] # instruction count
                instr_name = key.split('-')[0]
                time_instr += (instruction_count(instr_name, instr_vlen))*instr_cnt
            cgra_time[i] = (int(packet_length[i]/(SIMD_DEGREE*SIMD_under_utilization)) + time_instr) *PERIOD_CLK *1e-9 # in ns
    return cgra_time

In [34]:
import os
import numpy as np
verbose = False
total_rank = 96
num_iter = 5
TM = 1
TK = 2
vmacc_unroll_factor = 14
vmacc_vlen = 1354
SIMD_under_utilization = 1.0

def get_results(total_rank, num_iter, TM, TK, vmacc_unroll_factor, vmacc_vlen, SIMD_under_utilization)
    datasets = ['Cora', 'Citeseer', 'Pubmed', 'Reddit']
    # first element is vlen and second element is instruction count
    instruction = {'addi-0':[0, 5],
                   'lui-0':[0, 1],
                   'wfi-0':[0, 1],
                   # outer loop
                   'vsetivli-0':[0, 2*TM],
                   'vle32_v-0':[vmacc_unroll_factor, TM],
                   'addi-1':[0, 2*TM],
                   'bne-0':[0, TM],
                   'vstreamout_v-0':[vmacc_unroll_factor, TM],
                   # inner loop
                   'vsetivli-1':[0, 2*TM*TK],
                   'vle32_v-1':[vmacc_vlen, TM*TK],
                   'add-0':[0, TM*TK],
                   'addi-2':[0, TM*TK],
                   'bne-1':[0, TM*TK],
                   'spvmacc_xv-0':[vmacc_vlen, TM*TK*vmacc_unroll_factor]}

    # This is the raw data, storing all of the ranks
    my_dict_cpu_unroll = {datasets[0] + '_L1_XW':[[] for i in range(3*num_iter)], # tot_time_iter1, ..., tot_time_iter5, comm_iter1-5, ..., comp_iter1-5
               datasets[0] + '_L1_AXW':[[] for i in range(3*num_iter)],
               datasets[0] + '_L2_XW':[[] for i in range(3*num_iter)],
               datasets[0] + '_L2_AXW':[[] for i in range(3*num_iter)],
               datasets[1] + '_L1_XW':[[] for i in range(3*num_iter)],
               datasets[1] + '_L1_AXW':[[] for i in range(3*num_iter)],
               datasets[1] + '_L2_XW':[[] for i in range(3*num_iter)],
               datasets[1] + '_L2_AXW':[[] for i in range(3*num_iter)],
               datasets[2] + '_L1_XW':[[] for i in range(3*num_iter)],
               datasets[2] + '_L1_AXW':[[] for i in range(3*num_iter)],
               datasets[2] + '_L2_XW':[[] for i in range(3*num_iter)],
               datasets[2] + '_L2_AXW':[[] for i in range(3*num_iter)],
               datasets[3] + '_L1_XW':[[] for i in range(3*num_iter)],
               datasets[3] + '_L1_AXW':[[] for i in range(3*num_iter)],
               datasets[3] + '_L2_XW':[[] for i in range(3*num_iter)],
               datasets[3] + '_L2_AXW':[[] for i in range(3*num_iter)]}

    # This is the actual cpu result (max of all ranks)
    my_dict_cpu = {datasets[0] + '_L1_XW':[0 for i in range(num_iter)], # tot_time_iter1, ..., tot_time_iter5, comm_iter1-5, ..., comp_iter1-5
               datasets[0] + '_L1_AXW':[0 for i in range(num_iter)],
               datasets[0] + '_L2_XW':[0 for i in range(num_iter)],
               datasets[0] + '_L2_AXW':[0 for i in range(num_iter)],
               datasets[1] + '_L1_XW':[0 for i in range(num_iter)],
               datasets[1] + '_L1_AXW':[0 for i in range(num_iter)],
               datasets[1] + '_L2_XW':[0 for i in range(num_iter)],
               datasets[1] + '_L2_AXW':[0 for i in range(num_iter)],
               datasets[2] + '_L1_XW':[0 for i in range(num_iter)],
               datasets[2] + '_L1_AXW':[0 for i in range(num_iter)],
               datasets[2] + '_L2_XW':[0 for i in range(num_iter)],
               datasets[2] + '_L2_AXW':[0 for i in range(num_iter)],
               datasets[3] + '_L1_XW':[0 for i in range(num_iter)],
               datasets[3] + '_L1_AXW':[0 for i in range(num_iter)],
               datasets[3] + '_L2_XW':[0 for i in range(num_iter)],
               datasets[3] + '_L2_AXW':[0 for i in range(num_iter)]}

    my_dict_fpga = {datasets[0] + '_L1_AXW':[0 for i in range(num_iter)],
               datasets[0] + '_L2_AXW':[0 for i in range(num_iter)],
               datasets[1] + '_L1_AXW':[0 for i in range(num_iter)],
               datasets[1] + '_L2_AXW':[0 for i in range(num_iter)],
               datasets[2] + '_L1_AXW':[0 for i in range(num_iter)],
               datasets[2] + '_L2_AXW':[0 for i in range(num_iter)],
               datasets[3] + '_L1_AXW':[0 for i in range(num_iter)],
               datasets[3] + '_L2_AXW':[0 for i in range(num_iter)]}

    # assign directory
    fd = './GCN_results/96/'  
    for filenm in os.listdir(fd): # iterate over files in that directory
        file_name = os.path.join(fd, filenm)
        splitted_file_name = filenm.split("_")
        dataset = splitted_file_name[1]
        layer = splitted_file_name[2]
        iteration = int(splitted_file_name[3])
        tot_rank = int(splitted_file_name[4].split(".")[0])
        if iteration != 1: # ignore the first iteration
            with open(file_name) as f:
                f.readline() # ignore the first line
                f.readline() # ignore the second line
                lines = f.readlines()
                for i in range(tot_rank): # xW
                    splitted_lines = lines[i].split(" ")
                    rank = int(splitted_lines[0])
                    tot_time = float(splitted_lines[2])
                    comm_time = float(splitted_lines[3])
                    comp_time = float(splitted_lines[4].strip())
                    key = dataset + '_' + layer + '_' + 'XW' 
                    my_dict_cpu_unroll[key][(iteration-2)].append(tot_time) # -2 b/c the first valid iteration is 2
                    my_dict_cpu_unroll[key][(iteration-2) + num_iter].append(comm_time)
                    my_dict_cpu_unroll[key][(iteration-2) + num_iter*2].append(comp_time)
    #                 if verbose == True:
    #                     print('rank: ', rank)
    #                     print('tot_time: ', tot_time) # tot_time
    #                     print('comm_time: ', comm_time) # comm
    #                     print('comp_time: ', comp_time) # comp
                for i in range(tot_rank): # A(xW)
                    j = i + tot_rank
                    splitted_lines = lines[j].split(" ")
                    rank = int(splitted_lines[0])
                    tot_time = float(splitted_lines[2])
                    comm_time = float(splitted_lines[3])
                    comp_time = float(splitted_lines[4].strip())
                    key = dataset + '_' + layer + '_' + 'AXW' 
                    my_dict_cpu_unroll[key][(iteration-2)].append(tot_time) # -2 b/c the first valid iteration is 2
                    my_dict_cpu_unroll[key][(iteration-2) + num_iter].append(comm_time)
                    my_dict_cpu_unroll[key][(iteration-2) + num_iter*2].append(comp_time)
    #                 if verbose == True:
    #                     print('rank: ', rank)
    #                     print('tot_time: ', tot_time) # tot_time
    #                     print('comm_time: ', comm_time) # comm
    #                     print('comp_time: ', comp_time) # comp
                        # Automatically ignore the last line.

    # now my_dict_cpu_unroll is ready!
    # Lets calculate my_dict_cpu and my_dict_fpga
    dict_packet_length, ratio_diag = get_packetLength_per_rank(total_rank) # This is the number of elements
    for key in my_dict_cpu_unroll:
        result_list = my_dict_cpu_unroll[key]
        time_stamp = [[0 for h in range(total_rank)] for g in range(num_iter)]
        print(key)
        for j in range(3*num_iter):
            if key[-3:] == 'AXW': # otherwise if its XW we will not accelerate it
                if j < num_iter: # tot time calculate max(tot_time)
                    max_result = max(result_list[j])
                    my_dict_cpu[key][j%num_iter] = max_result * 1000.0 # in ms
                elif j >= num_iter and j < 2*num_iter: # comm (we get the timestamp from here)
                    max_result = max(result_list[j])
                    time_stamp[j%num_iter] = max_result - np.array(result_list[j]) # save time_stamp for later
                elif j >= 2*num_iter and j < 3*num_iter: # comp
                    this_ratio_diag = np.array(ratio_diag[key])
                    this_packet_length = np.array(dict_packet_length[key])
                    this_result_list = np.array(result_list[j])
                    result_comp_eff = this_result_list * this_ratio_diag # only diagonal
                    max_result = max(result_comp_eff * 1000.0) # in ms 
                    min_result = min(result_comp_eff * 1000.0) # in ms
                    packet_length = this_packet_length * (1.0 - this_ratio_diag) # only off-diagonal # This is the number of elements
    #                 time_stamp = np.array(result_list[j]) - min_result
                    gcn_fpga_time = gcn_FPGA_time_cgra(instruction, packet_length, SIMD_under_utilization)# depends on the rank b/c different ranks have different packet length
                    # gcn_fpga_time is a list but switch_time_global is a single time
                    switch_time_global, switch_time_port = comp_delay(time_stamp[j%num_iter], packet_length, total_rank, 'gcn', gcn_fpga_time)
                    tot_fpga_time = max((result_comp_eff * 1000.0) + (np.array(switch_time_port) * 1e-6)) # in ms
                    # 1000 and 1e-6 are for converting sec to ms and ns to ms respectively
                    my_dict_fpga[key][j%num_iter] = tot_fpga_time # ms
    #                 if verbose == True:
    #                     print('max comp fpga: ', max_result)
    #                     print('min comp fpga: ', min_result)
            else: # XW
                if j < num_iter: # tot time calculate max(tot_time)
                    max_result = max(result_list[j])
                    my_dict_cpu[key][j%num_iter] = max_result * 1000.0 # in ms

    # now my_dict_cpu and my_dict_fpga are ready
    print('--------- final result -----------')
    print('cpu results: ', my_dict_cpu)
    print('fpga results: ', my_dict_fpga)
    

GCN_matrices/Cora_feat_L2.txt
GCN_matrices/Cora_A.txt
GCN_matrices/Cora_feat_L1.txt
GCN_matrices/Pubmed_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
GCN_matrices/Cora_feat_L2.txt
GCN_matrices/Cora_A.txt
GCN_matrices/Cora_feat_L1.txt
GCN_matrices/Pubmed_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
GCN_matrices/Cora_feat_L2.txt
GCN_matrices/Cora_A.txt
GCN_matrices/Cora_feat_L1.txt
GCN_matrices/Pubmed_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Red

GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
GCN_matrices/Cora_feat_L2.txt
GCN_matrices/Cora_A.txt
GCN_matrices/Cora_feat_L1.txt
GCN_matrices/Pubmed_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
GCN_matrices/Cora_feat_L2.txt
GCN_matrices/Cora_A.txt
GCN_matrices/Cora_feat_L1.txt
GCN_matrices/Pubmed_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
GCN_matrices/Cora_feat_L2.txt
GCN_matrices/Cora_A.txt
GCN_matrices/Cora_feat_L1.txt
GCN_matrices/Pubmed_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices

In [44]:
import os
directory = './GCN_results/'
for filename in os.listdir(directory):
    fd = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isdir(fd) and fd.split('/')[-1] != '.ipynb_checkpoints':
        tot_rank = int(fd.split('/')[-1])
        print(fd, tot_rank)

./GCN_results/192 192
./GCN_results/1536 1536
./GCN_results/1152 1152
./GCN_results/384 384
./GCN_results/96 96
./GCN_results/768 768
